In [40]:
] st

      Status `~/Documents/projects/math_animations/roses/Project.toml`
  [d6d074c3] VideoIO v1.0.1
  [0f1e0344] WebIO v0.8.17


In [2]:
parentmodule(drawbezierpath)

Luxor

In [57]:
using ResumableFunctions
using Javis, Animations
using Colors

In [2]:
function reduceDenominator(numerator::Number, denominator::Number)::Number 
    function rec(a::Number, b::Number)::Number
        return b>0 ? rec(b, a % b) : a;
    end
    return denominator / rec(numerator, denominator);
end

reduceDenominator (generic function with 1 method)

In [3]:
@resumable function rose(n, d; step=0.1, a=1, pitimes=1)
    for i in 0:step:(reduceDenominator(n, d)*π*pitimes)
        r = a * cos(n / d * i);
        x = r * cos(i);
        y = r * sin(i);
        @yield [x, y];
    end
end

rose (generic function with 1 method)

In [4]:
function ground(args...)
    background("black")
    sethue("white")
end

ground (generic function with 1 method)

In [5]:
function circle_with_blend(pos, radius, colorblend)
    setblend(colorblend)
    circle(pos, radius, :fill)
    return pos
end

circle_with_blend (generic function with 1 method)

In [6]:
function circle_with_color(pos, radius, color)
    sethue(color)
    circle(pos, radius, :fill)
    return pos
end

circle_with_color (generic function with 1 method)

In [68]:
function path!(points, pos, colorblend)
    if(length(points)>0)
        if(! (last(points)==pos))
            push!(points, pos) # add pos to points
        end
    else
        push!(points, pos) 
    end
    setblend(colorblend)
    setline(3)
    drawbezierpath(makebezierpath(points), :stroke; close=false) 
end

path! (generic function with 1 method)

In [69]:
# the safest option is to declare the Video first all the time
begin
    video = Video(720, 1080)
    n=7
    d=4
    colora = "lightblue1"
    colorb = "gold"
    colorblend = blend(Point(-100, 0), Point(100, 0), colora, colorb)
    #video = Video(500, 500)
    
    # the points that describe our car
    points = [Point(v...) for v in rose(n,d; a=200, step=0.00001, pitimes=2)]
    npoints = length(points)

    Background(1:360, ground)
    
    # generate the bezier path
    bezierpath = makebezierpath(points)
    bezierpathpoly = bezierpathtopoly(bezierpath)
    
    
    # let a red circle appear and follow the bezier path polygon
    path = Point[]
    red_circle = Object(10:360, (args...) -> circle_with_blend(O, 7, colorblend), first(points))
    act!(red_circle, Action(1:20, appear(:fade)))
    act!(red_circle, Action(21:290, sineio(), follow_path(bezierpathpoly .- first(points))))
    act!(red_circle, Action(291:300, disappear(:fade)))

    
    curve = Object(20:360, (args...)->path!(path, pos(red_circle), colorblend))
    act!(curve, Action(320:340, disappear(:fade)))

    
    name = "rose_$colora-colorb_$n-$d.gif"
    render(video, pathname=name)
    display(MIME("text/html"), """<img src="$name?$(time())">""")
end

Rendering frames... 100%|████████████████████████████████| Time: 0:00:07
